In [1]:
import pandas as pd 
import kang
from tqdm import tqdm

In [2]:
ingff  =  './merged.gff3'
df_gff = pd.read_csv(ingff,comment='#',sep='\t',header=None)

In [4]:
startnum = 0
dic      = {}
new_col  = []
for ix in tqdm(df_gff.index):
    edf  = df_gff.loc[ix]
    scaf = edf[0] 
    if edf[2] == 'gene': ## ID=g1
        geneID = scaf+'_'+kang.infoparse(edf[8])['ID']
        newgeneID = 'Waus{:05d}g'.format(int(scaf.replace('scaffold',''))) + '{:05d}'.format(startnum)
        dic[geneID] = newgeneID 
        new_col.append('ID='+newgeneID)
        startnum += 1
    elif edf[2] == 'mRNA': ## ID=g1.t1;Parent=g1
        geneID          = scaf+'_'+kang.infoparse(edf[8])['Parent']
        newgeneID       = dic[geneID]
        transcriptID    = scaf+'_'+kang.infoparse(edf[8])['ID']
        newtranscriptID = newgeneID+'.'+transcriptID.split('.')[1]
        dic[transcriptID] = newtranscriptID 
        new_col.append('ID='+newtranscriptID+';'+'Parent='+newgeneID)
    elif edf[2] == 'CDS': ## ID=g1.t1.cds;Parent=g1.t1
        transcriptID       = scaf+'_'+kang.infoparse(edf[8])['Parent']
        newtranscriptID    = dic[transcriptID]
        cdsID              = scaf+'_'+kang.infoparse(edf[8])['ID']
        newcdsID           = newtranscriptID+'.'+cdsID.split('.')[-1]
        new_col.append('ID='+newcdsID+';'+'Parent='+newtranscriptID)

100%|██████████| 280660/280660 [02:09<00:00, 2175.51it/s]


In [6]:
m = df_gff[2].isin(['gene','mRNA','CDS'])
df_gff_sub = df_gff[m]
df_gff_sub[9] = new_col
df_gff_sub[[0,1,2,3,4,5,6,7,9]].to_csv(ingff+'.corrected.gff3',sep='\t',index=False,header=None)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
